In [76]:
# Dataset : https://www.kaggle.com/datasets/gauravduttakiit/ants-bees?resource=download
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader

In [77]:
transforms={'train':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),
    'val':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [78]:
data_dir='data/archive/hymenoptera_data'
#image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),transform=transforms[x]) for x in ['train','val']}


In [79]:
image_datasets={}
image_datasets['train']=datasets.ImageFolder(os.path.join(data_dir,'train'),transform=transforms['train'])
image_datasets['val']=datasets.ImageFolder(os.path.join(data_dir,'val'),transform=transforms['val'])

In [80]:
data_loader={}
data_loader['train']=DataLoader(image_datasets['train'],batch_size=32,shuffle=True)
data_loader['val']=DataLoader(image_datasets['val'],batch_size=32,shuffle=True)

In [81]:
model=models.resnet18(pretrained=True)
#print(model)
model_ftrs=model.fc.in_features
model.fc = nn.Sequential()
model.fc.ft1=nn.Linear(model_ftrs,out_features=256)
model.fc.ft2=nn.Linear(256,out_features=64)
model.fc.ft3=nn.Linear(64,out_features=16)
model.fc.ft4=nn.Linear(16,out_features=2)

criterian=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.fc.parameters(),lr=0.0001)


AttributeError: 'ResNet' object has no attribute 'layer5'

In [ ]:
for epoch in range(10):
    for input,label in data_loader['train']:
        output=model(input)
        loss=criterian(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(loss.item())


0.2412351667881012
0.7155871391296387
0.2865902781486511
0.2729128897190094
0.19802109897136688
0.3224591314792633
0.403866708278656
0.15577849745750427
0.22451892495155334
0.05855301022529602
0.2314821034669876
0.4219646453857422
0.17597076296806335
0.44653263688087463
0.36782604455947876
0.2533717751502991
0.16958613693714142
0.2732659876346588
0.18763147294521332
0.3339318633079529
0.13108229637145996
0.1542813628911972
0.31785154342651367
0.053596705198287964
0.15164229273796082
0.10341578722000122
0.18130168318748474
0.3991762697696686
0.3587201237678528
0.5275818109512329


In [ ]:
model.eval()
with torch.no_grad():
    correct=0
    total=0
    for input,label in data_loader['val']:
        output=model(input)
        _,predicted=torch.max(output.data,1)
        total+=label.size(0)
        correct+=(predicted==label).sum().item()
    print('Accuracy: {}%'.format(100*correct/total))
    print(total, correct)

Accuracy: 88.23529411764706%
153 135
